In [1]:
import pandas as pd

In [2]:
# CSVs laden
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")
projekte_df = pd.read_csv("../data/projekte.csv")

In [3]:
# Kompetenzspalte vorbereiten
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(", ")

In [4]:
# Matching-Funktion
matching_ergebnisse = []

for _, ta in teilaufgaben_df.iterrows():
    ta_start = int(ta["start"].split("/")[0])
    ta_ende = int(ta["ende"].split("/")[0])
    kandidaten = []

    for _, person in personen_df.iterrows():
        if ta["kompetenz"] not in person["kompetenzen_liste"]:
            continue

        # Verfügbarkeit über Zeitfenster aufsummieren
        verfuegbarkeit_summe = 0
        for monat in range(ta_start, ta_ende + 1):
            col = f"verfuegbarkeit_{monat:02d}/2025"
            if col in personen_df.columns:
                verfuegbarkeit_summe += person[col]
                
        score = round(verfuegbarkeit_summe * person["zeitbudget"], 2)

        kandidaten.append({
            "projekt_id": ta["projekt_id"],
            "teilaufgabe_id": ta["teilaufgabe_id"],
            "teilaufgabe": ta["bezeichnung"],
            "kompetenz": ta["kompetenz"],
            "person_id": person["id"],
            "name": person["name"],
            "score": score,
            "verfuegbarkeit_summe": round(verfuegbarkeit_summe, 2)
        })

    top_kandidaten = sorted(kandidaten, key=lambda x: x["score"], reverse=True)[:3]
    matching_ergebnisse.extend(top_kandidaten)


In [5]:
# Ergebnis speichern
pd.DataFrame(matching_ergebnisse).to_csv("matching_ergebnis.csv", index=False)
print("Matching abgeschlossen – Ergebnisse gespeichert in 'matching_ergebnis.csv'")

Matching abgeschlossen – Ergebnisse gespeichert in 'matching_ergebnis.csv'
